***Syntax natürlicher Sprachen, WS 2021/22***

---
# Übung 2 (Lösung)

---
## Aufgabe 1:  Eine erste syntaktische Analyse

#### Kreieren Sie Ihre ersten Syntaxbäume, indem Sie das folgende Python-Skript ausführen. Beantworten Sie im Anschluss die zugehörigen Fragen!

In [6]:
import nltk

grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

parser = nltk.ChartParser(grammar)
sent = "I shot an elephant in my pajamas".split()
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
     S                                       
 ┌───┴──────────────┐                         
 │                  VP                       
 │        ┌─────────┴──────────┐              
 │        VP                   PP            
 │   ┌────┴───┐            ┌───┴───┐          
 │   │        NP           │       NP        
 │   │    ┌───┴─────┐      │   ┌───┴─────┐    
 NP  V   Det        N      P  Det        N   
 │   │    │         │      │   │         │    
 I  shot  an     elephant  in  my     pajamas

(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))
     S                                   
 ┌───┴──────────┐                         
 │              VP                       
 │   ┌──────────┴──────┐                  
 │   │                 NP                
 │   │    ┌─────┬──────┴───┐              
 │   │    │     │       

### Fragen zu Aufgabe 1

---
#### Aufgabe 1 a: Warum gibt die parse-Funktion des Skripts vermutlich eine Sequenz zurück?

Ambiguität! Daher ist im Allgemeinen mehr als nur ein Parse-Tree möglich.


---
#### Aufgabe 1b: Worin liegen die Unterschiede zwischen den ausgegebenen Bäumen?

Die Präpositionalphrase „in my pajamas“ ist einmal Teil der Verbalphrase und einmal Teil der Nominalphrase.

___
#### Aufgabe 1 c: Warum ist der gegebene Satz "I shot an elephant in my pajamas" syntaktisch ambig?

Aus der Oberflächensyntax ist nicht ersichtlich, welche syntakische Funktion die PP in my pajamas einnimmt. Handel es sich um ein Adverbial oder um ein Attribut? Ist Ersteres der Fall, modifiziert die PP die im Verb ausgedrückete Tätigkeit. Im zweiten Fall stellt die PP eine Beifügung zum Substantiv dar.



---
## Aufgabe 2: Dependenzgrammatik

#### Schreiben Sie zu dem Satz aus Aufgabe 1 eine Dependenzgrammatik, die die syntaktische Ambiguität der Satzanalyse erhält.

#### Verwenden Sie dabei die *Primacy-of-Content-Words*-Maxime des Universal-Dependencies-Schemas für die PP-Analyse (https://universaldependencies.org/u/overview/syntax.html#the-primacy-of-content-words).

In [2]:
grammar = nltk.DependencyGrammar.fromstring("""
    'shot' -> 'I' | 'elephant' | 'pajamas'
    'elephant' -> 'an' | 'pajamas'
    'pajamas' -> 'in' | 'my'
    """)

parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(shot I (elephant an) (pajamas in my))
      shot                  
 ┌─────┼────────────┐        
 │  elephant     pajamas    
 │     │      ┌─────┴─────┐  
 I     an     in          my

(shot I (elephant an (pajamas in my)))
        shot                     
 ┌───────┴──────┐                 
 │           elephant            
 │   ┌──────────┴────────┐        
 │   │                pajamas    
 │   │          ┌────────┴─────┐  
 I   an         in             my



---
## Aufgabe 3: Ambiguität

#### Sehen Sie sich die folgenden englischen Sätze an. Können Sie sich jeweils zwei unterschiedliche Interpretationen vorstellen?

- *Fighting animals could be dangerous.*
- *Visiting relatives can be tiresome.*


#### Erweitern Sie die gegebene Grammatik, so dass für den zweiten Satz linguistisch korrekte Syntaxanalysen beider Interpretationen ausgegeben werden.

- `VBG`: *Verb, gerund or present participle* (Penn Treebank)
- `SBAR`: *Embedded subordinate clause* (Penn Treebank)

In [7]:
sent = 'visiting relatives can be tiresome'.split()

In [8]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    S -> SBAR VP
    NP -> VBG N | N
    SBAR -> VP    
    VP -> AUX VP | COP ADJ | VBG NP
    VBG -> 'visiting'
    N -> 'relatives'
    AUX -> 'can'
    COP -> 'be'
    ADJ -> 'tiresome'
""")
    
parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (VBG visiting) (N relatives))
  (VP (AUX can) (VP (COP be) (ADJ tiresome))))
                        S                  
          ┌─────────────┴───┐               
          │                 VP             
          │             ┌───┴───┐           
          NP            │       VP         
   ┌──────┴──────┐      │   ┌───┴─────┐     
  VBG            N     AUX COP       ADJ   
   │             │      │   │         │     
visiting     relatives can  be     tiresome

(S
  (SBAR (VP (VBG visiting) (NP (N relatives))))
  (VP (AUX can) (VP (COP be) (ADJ tiresome))))
                         S                  
          ┌──────────────┴───┐               
         SBAR                │              
          │                  │               
          VP                 VP             
   ┌──────┴───────┐      ┌───┴───┐           
   │              NP     │       VP         
   │              │      │   ┌───┴─────┐     
  VBG             N     AUX COP       ADJ   
   │  

---
### Fragen zu Aufgabe 3

---
#### Was ist der Ursprung der Ambiguität?  (Lexikalische Ambiguität der einzelnen Wörter? Morphologisch ambige Formen? Andere Gründe?)


Weder Einzelwörter noch Formen sind (im Kontext) ambig. Die Ambiguität liegt in der Beziehung zwischen Gerund (*fighting*, *visiting*) und nachfolgendem Nomen:

1. Subjekt: Kämpfende Tiere / Verwandte, die zu Besuch sind
2. direktes Objekt: Tiere zu bekämpfen / Verwandte zu besuchen

Es liegt also eine strukturelle Ambiguität vor: im ersten Fall ist das Nomen Kopf einer NP-Phrase, im zweiten Fall ist das Gerund Kopf eines nichtfiniten Teilsatzes.


---
#### In welcher Beziehung können Gerund und nachfolgendes Nomen stehen?

1. Gerund ist Dependent von Nomen als Kopf einer NP-Phrase
2. Nomen ist Dependent von Gerund als Kopf eines nichtfiniten Teilsatzes


## Analyse der beiden Varianten mit SpaCy:

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy import displacy

In [4]:
doc = nlp('Visiting relatives can be tiresome.')
displacy.render(doc, style="dep", options={'distance':100})

# Nomen ist Kopf, Gerund ist Dependent (adjektivisches Attribut)

In [5]:
doc = nlp('Seeing relatives can be tiresome.')
displacy.render(doc, style="dep", options={'distance':100})

# Gerund ist Kopf, Nomen ist Dependent (direktes Objekt)


---
#### Welche Funktion im Satz nimmt das Nomen ein, wenn es Kopf der Gerund-Nomen-Konstituente ist?

Subjekt (`nsubj`)

---
#### Welche Funktion innerhalb der Gerund-Nomen-Konstituente nimmt das Nomen ein, wenn das Gerund Kopf dieser Phrase ist?

Objekt (`dobj`)

---
#### Welche Funktion im Hauptsatz nimmt das Gerund ein, wenn es Kopf der Gerund-Nomen-Konstituente ist?

Subjekt (`csubj`)

---
#### Welche Funktion nimmt das Gerund ein, wenn das Nomen Kopf der Gerund-Nomen-Konstituente ist?

Attribut (`amod`)

---
# Hausaufgaben

---
## Aufgabe 4: Darstellung syntaktischer Strukturen

---
### Aufgabe 4 a:
 
#### Geben Sie zu folgendem Syntaxbaum den Klammerausdruck an (für den Fall, dass der Elefant den Pyjama trägt):

In [5]:
from nltk import Tree
tree = Tree.fromstring("""
(I-shot-an_elephant-in_my_pajamas
  I
  (shot-an_elephant-in_my_pajamas
    shot
    (an_elephant-in_my_pajamas
      an_elephant
      in_my_pajamas
    )
  )
)
""")
tree.pretty_print(unicodelines=True)

         I-shot-an_elepha                                              
         nt-in_my_pajamas                                              
 ┌──────────────┴─────────────────┐                                     
 │                        shot-an_elephant-                            
 │                          in_my_pajamas                              
 │   ┌────────────────────────────┴───────────────┐                     
 │   │                                       an_elephant-              
 │   │                                      in_my_pajamas              
 │   │                            ┌───────────────┴─────────────┐       
 I  shot                     an_elephant                  in_my_pajamas



In [6]:
# hier den Klammerausdruck angeben:

tree = Tree.fromstring("""
(ROOT
    (I) ((shot) ((an_elephant) (in_my_pajamas)))
)
""")

tree.pretty_print(unicodelines=True)

         ROOT                              
 ┌────────┴────────┐                        
 │                                         
 │   ┌─────────────┴───────┐                
 │   │                                     
 │   │             ┌───────┴────────┐       
 I  shot      an_elephant     in_my_pajamas
 │   │             │                │       
... ...           ...              ...     



---
### Aufgabe 4 b:

#### Geben Sie zu folgendem Klammerausdruck den Syntaxbaum an (wer trägt hier den Pyjama?)


In [9]:
#[ [I] [ [ [shot] [an elephant] ] [in my pajamas] ] ]

#### Klammern Sie dazu zunächst den Klammerausdruck aus 3a um:

In [7]:
# hier von 3a herunterkopieren und umklammern:

tree = Tree.fromstring("""
(ROOT
(I) (((shot) (an_elephant)) (in_my_pajamas))
)
""")

tree.pretty_print(unicodelines=True)

         ROOT                          
 ┌────────┴────────┐                    
 │                                     
 │        ┌────────┴────────────┐       
 │                              │      
 │   ┌────┴────────┐            │       
 I  shot      an_elephant in_my_pajamas
 │   │             │            │       
... ...           ...          ...     



#### Geben Sie dann den entsprechende Syntaxbaum an, indem Sie den Satz in seine Subkonstituenten zerlegen:

In [8]:
# hier den Syntaxbaum angeben:

tree = Tree.fromstring("""
(I-shot-an_elephant-in_my_pajamas
    (I)
    (shot-an_elephant-in_my_pajamas
        (shot-an_elephant
            (shot)
            (an_elephant)
        )
        (in_my_pajamas)
    )
)
""")

tree.pretty_print(unicodelines=True)

         I-shot-an_elepha                                
         nt-in_my_pajamas                                
 ┌──────────────┴─────────────────┐                       
 │                        shot-an_elephant-              
 │                          in_my_pajamas                
 │              ┌─────────────────┴───────────────┐       
 │       shot-an_elephant                         │      
 │   ┌──────────┴─────────────────┐               │       
 I  shot                     an_elephant    in_my_pajamas
 │   │                            │               │       
... ...                          ...             ...     

